# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
indeed = requests.get(URL)

In [4]:
for k, v in indeed.request.headers.items():
    print k + ':', v

Connection: keep-alive
Cookie: INDEED_CSRF_TOKEN=UDSJ7QtKwRyxAAjcFTmT79Q9INzNaHW2; ctkgen=1; BIGipServerjob=!puUJvMTioDcwSiY/0rl98CFw+W0yBo/g72SHhMbXOM14i+wKoFCSVgAgU+LJVomFWc/B+b08zvgGtV0=; JSESSIONID=387DAB66ABE834912D2BA9BF2162B43C.jasxA_ord-job16; CTK=1amtlt7hgagm2f7i
Accept-Encoding: gzip, deflate
Accept: */*
User-Agent: python-requests/2.9.1


In [5]:
soup = BeautifulSoup(indeed.content, "lxml")

In [6]:
count = 0
for i in soup.findAll('div', class_ = '  row result'):
    print count, i.text
    print '\n'
    count += 1

0 

Director of Data Science and Analytics



    Paperless Post


 - New York, NY





Decision science and data analytics. You’ll lead a team of six data scientists, developing a roadmap of initiatives that improve our decision-making, business...


 Easily apply

3 days ago  - save job  -  email - more...window['result_b12d1031ae3c0d25'] = {"showSource": false, "source": "Paperless Post", "loggedIn": false, "showMyJobsLinks": true,"undoAction": "unsave","relativeJobAge": "3 days ago","jobKey": "b12d1031ae3c0d25", "myIndeedAvailable": true, "tellAFriendEnabled": true, "showMoreActionsLink": true, "resultNumber": 0, "jobStateChangedToSaved": false, "searchState": "q=data scientist $20,000&amp;l=New+York", "basicPermaLink": "http://www.indeed.com", "saveJobFailed": false, "removeJobFailed": false, "requestPending": false, "notesEnabled": true, "currentPage" : "serp", "sponsored" : false,"reportJobButtonEnabled": false, "showMyJobsHired": false};

View all Paperless Post jobs in New Yor

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above

In [7]:
def get_jobtitle(result):
    jobtitle_list = []
    for i in result.findAll('a', class_ = 'jobtitle turnstileLink'): 
        jobtitle_list.append(str(i.text))
    return jobtitle_list

def get_location(result):
    location_list = []
    for i in result.findAll('span', class_ = 'location'):
        location_list.append(str(i.text))
    return location_list

def get_company(result):        
    company_list = []
    for i in soup.findAll('span', class_ = 'company'):
        company = str(i.text)
        company = company.replace('\n', '')
        company = company.replace('    ', '')
        company_list.append(company)
    return company_list
    
def get_salary(result):
    salary_list = []
    for i in soup.findAll('td', class_ = 'snip'):
        if i.find('nobr') != None:
            salary_list.append(i.find('nobr').text)
        else:
            salary_list.append('NaN')
    return salary_list

def putitalltogether(result):
    jobs = get_jobtitle(result)
    locations = get_location(result)
    companies = get_company(result)
    salaries = get_salary(result)
    return pd.DataFrame(zip(jobs, locations, companies, salaries),\
                        columns = ['title', 'location', 'company', 'salary']) 

In [8]:
putitalltogether(soup)

,title,location,company,salary
0,"Senior Epidemiologist, Bureau Systems Strength...","Queens, NY",DEPT OF HEALTH/MENTAL HYGIENE,NaN
1,Data Scientist,"New York, NY","AbilTo, Inc",NaN
2,Data Scientist & Analytics Developer,"Long Beach, NY",Lancer Insurance Company,NaN
3,Data Scientist - Monitoring & Anomaly Detectio...,"New York, NY",Paperless Post,NaN
4,Senior Scientist,"New York, NY",Bloomberg,NaN


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [10]:
YOUR_CITY = 'Rome'

In [30]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100

results = []
df = pd.DataFrame(columns=['jobtitle', 'location', 'company', 'salary'])
#df = df.fillna(0) # with 0s rather than NaNs

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        start_URL = 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l='
        mid_URL = '&start='
        URL = start_URL + city + mid_URL + str(start)
        indeed = requests.get(URL)
        print indeed
        soup = BeautifulSoup(indeed.content, "lxml")
        # Append to the full set of results
        x = putitalltogether(soup)
        #df = pd.concat(x, df)
#print df
    

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


UnicodeEncodeError: 'ascii' codec can't encode character u'\u2013' in position 15: ordinal not in range(128)

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [10]:
## YOUR CODE HERE

## Predicting salaries using Logistic Regression

#### Load in the data of scraped salaries

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('/Users/Rebecca/DSI-projects/curriculum/week04/5.2-lab-project-4-logreg/assets/datasets/indeed-scraped-job-postings.csv')

In [19]:
df.head(2)

,city,company,salary,summary,title,parsed_salary
0,San+Francisco,MarkMonitor,"$180,000 a year","Data skills (SQL, Hive, Pig). Applying machine...",Data Scientist,180000.0
1,San+Francisco,Workbridge Associates,"$130,000 - $180,000 a year",3+ years of industry experience in a data scie...,Senior Data Scientist,155000.0


#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [36]:
median = float(df.describe().loc['50%'])

In [39]:
def dummify(x):
    x['dummy'] = 0
    for i in range(len(x)):
        if x['parsed_salary'][i] > median:
            x['dummy'][i] = 1
        else:
            continue
    return x

In [ ]:
dummify()

#### Thought experiment: What is the baseline accuracy for this model?

In [16]:
## YOUR CODE HERE

#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [17]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [18]:
## YOUR CODE HERE

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [19]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [20]:
## YOUR CODE HERE

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [21]:
## YOUR CODE HERE

In [22]:
## YOUR CODE HERE

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [23]:
## YOUR CODE HERE

In [24]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [25]:
## YOUR CODE HERE